In [117]:
import pandas as pd
import numpy as np

In [118]:
index = pd.Index(pd.date_range(start="2025-05-01", end="2025-06-01", freq="min"), name="date")
index


df = pd.DataFrame(np.arange(44641), columns=["data"], index=index)

# select by time:
minute_after_midnight_rows = df.loc[df.index.time == pd.Timestamp("00:01:00").time()]
minute_after_midnight_rows


,data
date,
2025-05-01 00:01:00,1
2025-05-02 00:01:00,1441
2025-05-03 00:01:00,2881
2025-05-04 00:01:00,4321
2025-05-05 00:01:00,5761
2025-05-06 00:01:00,7201
2025-05-07 00:01:00,8641
2025-05-08 00:01:00,10081
2025-05-09 00:01:00,11521


In [119]:
# Another way using a mask
midnight_mask = df.index.time == pd.Timestamp("00:00:00").time()
df.loc[midnight_mask]

,data
date,
2025-05-01,0
2025-05-02,1440
2025-05-03,2880
2025-05-04,4320
2025-05-05,5760
2025-05-06,7200
2025-05-07,8640
2025-05-08,10080
2025-05-09,11520


In [120]:
ser = pd.Series(range(4), index=["aaa", "bbb", "ccc", "bbb"])

# This will throw an error that it cannot get the right slice bound...
# ...since the label "bbb" is not unique.
ser.loc[:"bbb"]

KeyError: "Cannot get right slice bound for non-unique label: 'bbb'"

In [121]:
# label-based selection from a dataframe
df = pd.DataFrame([
    [24, 180, "blue"],
    [42, 166, "brown"],
    [22, 160, "green"],
], columns=["age", "height_cm", "eye_color"], index=["Jack", "Jill", "Jayne"])
df

,age,height_cm,eye_color
Jack,24,180,blue
Jill,42,166,brown
Jayne,22,160,green


In [122]:
df.loc["Jayne", "eye_color"]

'green'

In [123]:
df.loc[:, "age"]

Jack     24
Jill     42
Jayne    22
Name: age, dtype: int64

In [124]:
df.loc["Jack", :]

age            24
height_cm     180
eye_color    blue
Name: Jack, dtype: object

In [125]:
df.loc[["Jack"], :]

,age,height_cm,eye_color
Jack,24,180,blue


In [126]:
df.loc[:, ["age"]]

,age
Jack,24
Jill,42
Jayne,22


In [127]:
df.loc[["Jack", "Jill"], ["age", "eye_color"]]

,age,eye_color
Jack,24,blue
Jill,42,brown


# Mixing position-based and label-based selection

In [128]:
df = pd.DataFrame([
    [24, 180, "blue"],
    [42, 166, "brown"],
    [22, 160, "green"],
], columns=["age", "height_cm", "eye_color"])
df

,age,height_cm,eye_color
0,24,180,blue
1,42,166,brown
2,22,160,green


## Indexer
Getting the index (position) of a column, given its label name

In [129]:
col_idxer = df.columns.get_indexer(["age", "eye_color"])
col_idxer

array([0, 2])

We can then use col_idxer as an argument to df.iloc as follows:

In [130]:
df.iloc[[0, 1], col_idxer] # selecting first two rows and columns by col_idxer

,age,eye_color
0,24,blue
1,42,brown


In [131]:
df[["age", "eye_color"]].iloc[[0, 1], :] # same result as above, but broken into steps

,age,eye_color
0,24,blue
1,42,brown


# Why bother with pd.Index.get_indexer?
Performance differences:

In [132]:
import timeit

def get_indexer_approach():
    col_idxer = df.columns.get_indexer(["age", "eye_color"])
    df.iloc[[0, 1], col_idxer]

timeit.timeit(get_indexer_approach, number = 10_000)

3.0986593079996965

In [133]:
# The two-step approach actually creates an intermediate df after df[["age", "eye_color"]]
two_step_approach = lambda: df[["age", "eye_color"]].iloc[[0, 1]]
timeit.timeit(two_step_approach, number = 10_000)

3.1771669939989806

# DataFrame.filter

In [134]:
df = pd.DataFrame([
    [24, 180, "blue"],
    [42, 166, "brown"],
    [22, 160, "green"],
], columns=[
    "age",
    "height_cm",
    "eye_color"
], index=["Jack", "Jill", "Jayne"])
df

,age,height_cm,eye_color
Jack,24,180,blue
Jill,42,166,brown
Jayne,22,160,green


In [135]:
df.filter(["age", "eye_color"])

,age,eye_color
Jack,24,blue
Jill,42,brown
Jayne,22,green


## Using the axis= argument
axis = 0 selects rows

axis = 1 selects columns

In [136]:
df.filter(["age", "eye_color"], axis = 1)

,age,eye_color
Jack,24,blue
Jill,42,brown
Jayne,22,green


In [137]:
df.filter(["Jack", "Jill"], axis = 0)

,age,height_cm,eye_color
Jack,24,180,blue
Jill,42,166,brown


In [138]:
# We can also use some pattern matching:
df.filter(like="_", axis = 1) # this selects column containing an underscore

,height_cm,eye_color
Jack,180,blue
Jill,166,brown
Jayne,160,green


In [139]:
df.filter(like="Ja", axis = 0) # selects rows with labels containing "Ja" substring

,age,height_cm,eye_color
Jack,24,180,blue
Jayne,22,160,green


## Regex= parameter
Example:

^: asserts that a match starts at the beginning of the string

Ja: matches the literal characters "Ja"

.*: matches zero or more of any character (except a newline)

[^e]: matches any character EXCEPT 'e' (ensures we don't end with 'e')

$: asserts that the match ends at the end of the string

In [140]:
df.filter(regex=r"^Ja.*(?<!e)$", axis = 0)

,age,height_cm,eye_color
Jack,24,180,blue


# Selecting by dtypes

In [141]:
df = pd.DataFrame([
    [0, 1.0, "2", pd.Timestamp("2025-05-01")],
    [4, 8.0, "16", pd.Timestamp("2025-06-01")],
], columns = ["int_col", "float_col", "string_col", "timestamp"])
df

,int_col,float_col,string_col,timestamp
0,0,1.0,2,2025-05-01
1,4,8.0,16,2025-06-01


In [142]:
# Let's select only the integer columns:
df.select_dtypes("int")

,int_col
0,0
1,4


In [143]:
# Select only the float columns:
df.select_dtypes("float")

,float_col
0,1.0
1,8.0


In [144]:
# "string" dtypes aren't allowed, so you have to select "object":
df.select_dtypes("object")

,string_col
0,2
1,16


In [145]:
df.select_dtypes(include=["datetime", "int"])

,int_col,timestamp
0,0,2025-05-01
1,4,2025-06-01


In [146]:
df.select_dtypes(exclude=["datetime"])

,int_col,float_col,string_col
0,0,1.0,2
1,4,8.0,16


# Selection/filtering via Boolean arrays

In [147]:
# Boolean lists/arrays are also called masks
mask = [True, False, True]
ser = pd.Series(range(3))
ser

0    0
1    1
2    2
dtype: int64

In [148]:
# The folowing should exclude the second element:
ser[mask]

0    0
2    2
dtype: int64

In [149]:
# same result above as using ser.loc
ser.loc[mask]

0    0
2    2
dtype: int64

In [150]:
df = pd.DataFrame(np.arange(6).reshape(3, -1))
df

,0,1
0,0,1
1,2,3
2,4,5


In [151]:
# This selects along ROWS
df[mask]

,0,1
0,0,1
2,4,5


In [152]:
# df.loc will accept TWO MASK arguments:
col_mask = [True, False]
df.loc[mask, col_mask]

,0
0,0
2,4


In [153]:
# Manipulating masks using OR, AND, or INVERT operators
df = pd.DataFrame([
    [24, 180, "blue"],
    [42, 166, "brown"],
    [22, 160, "green"],
], columns=["age", "height_cm", "eye_color"], index=["Jack", "Jill", "Jayne"])
df

,age,height_cm,eye_color
Jack,24,180,blue
Jill,42,166,brown
Jayne,22,160,green


In [154]:
# Filtering to users with blue or green eyes:
# step 1: filter by blue eyes
blue_eyes = df["eye_color"] == "blue"
blue_eyes

Jack      True
Jill     False
Jayne    False
Name: eye_color, dtype: bool

In [155]:
# step 2: filter green eyes
green_eyes = df["eye_color"] == "green"
green_eyes

Jack     False
Jill     False
Jayne     True
Name: eye_color, dtype: bool

In [156]:
# step 3: Then combine these using OR or | operator
mask = blue_eyes | green_eyes
mask

Jack      True
Jill     False
Jayne     True
Name: eye_color, dtype: bool

In [157]:
# step 4: finally, use this mask to get users with blue or green eyes
df[mask]

,age,height_cm,eye_color
Jack,24,180,blue
Jayne,22,160,green


In [158]:
# What about using combinations via AND or &?
# Let's find someone with an age less than 40 but a height greater than 170:

In [159]:
age_lt_40 = df["age"] < 40
age_lt_40

Jack      True
Jill     False
Jayne     True
Name: age, dtype: bool

In [160]:
height_gt_170 = df["height_cm"] > 170
height_gt_170

Jack      True
Jill     False
Jayne    False
Name: height_cm, dtype: bool

In [161]:
mask = age_lt_40 & height_gt_170
mask

Jack      True
Jill     False
Jayne    False
dtype: bool

In [162]:
df[mask]

,age,height_cm,eye_color
Jack,24,180,blue


In [163]:
# or just & them together in the df itself
df[age_lt_40 & height_gt_170]

,age,height_cm,eye_color
Jack,24,180,blue


In [164]:
# Note that we can invert the mask with ~
df[~mask]

,age,height_cm,eye_color
Jill,42,166,brown
Jayne,22,160,green


# MultiIndex

In [165]:
index = pd.MultiIndex.from_tuples([
    ("John", "Smith"),
    ("John", "Doe"),
    ("Jane", "Doe"),
    ("Stephen", "Smith"),
], names=["first_name", "last_name"])
ser = pd.Series(range(4), index=index)
ser

first_name  last_name
John        Smith        0
            Doe          1
Jane        Doe          2
Stephen     Smith        3
dtype: int64

In [166]:
ser.loc["John"]

last_name
Smith    0
Doe      1
dtype: int64

In [167]:
# to avoid level reduction in a multiindex slice:
ser.loc[["John"]]

first_name  last_name
John        Smith        0
            Doe          1
dtype: int64

# MultiIndex - multiple level selections

In [168]:
index = pd.MultiIndex.from_tuples([
    ("John", "Smith"),
    ("John", "Doe"),
    ("Jane", "Doe"),
    ("Stephen", "Smith"),
], names=["first_name", "last_name"])
ser = pd.Series(range(4), index=index)
ser

first_name  last_name
John        Smith        0
            Doe          1
Jane        Doe          2
Stephen     Smith        3
dtype: int64

In [169]:
ser.loc[("Jane", "Doe")]

np.int64(2)

In [170]:
# to maintain shape, place the first index level in a list:
ser.loc[(["Jane"], "Doe")]

first_name  last_name
Jane        Doe          2
dtype: int64

In [172]:
# select all records where first index uses "John" and second uses "Smith"
# OR
# first level uses "Jane" and second uses "Doe"
# note the two [[]]
ser.loc[[("John", "Smith"), ("Jane", "Doe")]]

first_name  last_name
John        Smith        0
Jane        Doe          2
dtype: int64